In [19]:
from datetime import datetime
import calendar
from collections import defaultdict

# Helper to calculate overlapping days
def get_overlap_days(start, end, month_start, month_end):
    overlap_start = max(start, month_start)
    overlap_end = min(end, month_end)
    delta = (overlap_end - overlap_start).days + 1
    return max(delta, 0), overlap_start, overlap_end

# ✅ Final Billing Function
def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    year, month = map(int, target_month.split('-'))
    month_start = datetime(year, month, 1)
    last_day = calendar.monthrange(year, month)[1]
    month_end = datetime(year, month, last_day)

    grouped_items = defaultdict(lambda: {"qty": 0, "amount": 0.0})

    for item in item_list:
        start_date = datetime.strptime(item["start_date"], "%Y-%m-%d")
        end_date = datetime.strptime(item["end_date"], "%Y-%m-%d") if item.get("end_date") else month_end

        overlap_days, overlap_start, overlap_end = get_overlap_days(start_date, end_date, month_start, month_end)

        if overlap_days > 0:
            item_code = item["item_code"]
            rate = item["rate"]
            qty = item["quantity"]
            days_in_month = (month_end - month_start).days + 1

            prorated_rate = (rate * overlap_days) / days_in_month
            amount = round(prorated_rate * qty, 2)

            billing_period = f"{overlap_start.date()} to {overlap_end.date()}"

            key = (item_code, rate, billing_period)

            grouped_items[key]["qty"] += qty
            grouped_items[key]["amount"] += amount

    line_items = []
    total_revenue = 0.0

    for (item_code, rate, billing_period), values in grouped_items.items():
        amount = round(values["amount"], 2)
        line_items.append({
            "item_code": item_code,
            "rate": float(rate),
            "qty": values["qty"],
            "amount": amount,
            "billing_period": billing_period
        })
        total_revenue += amount

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }


In [7]:
item_list = [
    {
        "item_code": "Executive Desk (4*2)",
        "rate": 1080.0,
        "quantity": 25,
        "start_date": "2024-11-01",
        "end_date": "2024-11-30"
    },
    {
        "item_code": "Manager Cabin",
        "rate": 5000.0,
        "quantity": 5,
        "start_date": "2024-11-05",
        "end_date": "2024-11-30"
    }
]

result = generate_monthly_bill(item_list, "2024-11")
print(result)


{'line_items': [{'item_code': 'Executive Desk (4*2)', 'rate': 1080.0, 'qty': 25, 'amount': 27000.0, 'billing_period': '2024-11-01 to 2024-11-30'}, {'item_code': 'Manager Cabin', 'rate': 5000.0, 'qty': 5, 'amount': 21666.67, 'billing_period': '2024-11-05 to 2024-11-30'}], 'total_revenue': 48666.67}
